In [21]:
simulation_data = {
    'type': 'REGULAR',
    'settings': {
        'instrumentType': 'EQUITY',
        'region': 'USA',
        'universe': 'TOP3000',
        'delay': 1,
        'decay': 0,
        'neutralization': 'INDUSTRY',
        'truncation': 0.08,
        'pasteurization': 'ON',
        'unitHandling': 'VERIFY',
        'nanHandling': 'OFF',
        'language': 'FASTEXPR',
        'visualization': False
    },
    'regular': 'liabilities/assets'
}

In [ ]:
import requests
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from os.path import expanduser
from requests.auth import HTTPBasicAuth
from time import sleep
import pandas as pd
from typing import Dict, List
from time import sleep

class BrainClient:
    
    def __init__(self, credential_file: str):
        self._pool = ThreadPoolExecutor(max_workers=3)
        with open(expanduser(credential_file)) as f:
            credentials = json.load(f)
            username, password = credentials["username"], credentials["password"]
        self._sess = requests.Session()
        self._sess.auth = HTTPBasicAuth(username, password)
        response = self._sess.post("https://api.worldquantbrain.com/authentication")
        print(response.status_code)
        print(response.json())

    def get_datafields(self, searchScope, dataset_id:str, search:str):
        instrument_type = searchScope['instrumentType']
        region = searchScope['region']
        delay = searchScope['delay']
        universe = searchScope['universe']
        if len(search) == 0:
            url_template = 'https://api.worldquantbrain.com/data-fields?' +\
                f'&instrumentType={instrument_type}' +\
                f'region={region}&delay={str(delay)}&universe={universe}&dataset.id={dataset_id}&limit=50&offset={x}'
            count = self._sess.get(url_template.format(x=0)).json()['count']
        else:
            url_template = 'https://api.worldquantbrain.com/data-fields?' +\
                f'&instrumentType={instrument_type}' +\
                f'region={region}&delay={str(delay)}&universe={universe}&search={search}&limit=50&offset={x}'
            count = 100
        datafield_list = []
        for x in range(0,count, 50):
            datafields = self._sess.get(url_template.format(x=x))
            datafield_list.append(datafields.json()["results"])
        datafield_list_flat = [item for sublist in datafield_list for item in sublist]
        datafields_df = pd.DataFrame(datafield_list_flat)
        return datafields_df

    def _generate_alpha(self, alpha_template:str, values_list: List[Dict[str, str]])->str:
        return [alpha_template.format_map(values) for values in values_list]

    def _simulate(self, simulation_data_list: List[Dict]):
        for simulation_data in simulation_data_list:
            sim_resp = sess.post(
                'https://api.worldquantbrain.com/simulations',
                json=simulation_data
            )
            print(sim_resp.status_code)
            if 'Location' not in sim_resp.headers:
                print(f"simulate error, simulation_data: {simulation_data}, response: {sim_resp.json()}") 
            sim_progress_url = sim_resp.headers['Location']
            
            while True:
                sim_progress_resp = self._sess.get(sim_progress_url)
                retry_after_sec = float(sim_progress_resp.headers.get('Retry-After', 0))
                if retry_after_sec == 0:
                    break
                sleep(retry_after_sec)
            
            alpha_id = sim_progress_resp.json()['alpha'] 
            print(alpha_id)

    def batch_simulate

    def wait_task_done(self):
        self._pool.shutdown(wait=True)

    


In [ ]:
simulation_data = {
    'type': 'REGULAR',
    'settings': {
        'instrumentType': 'EQUITY',
        'region': 'USA',
        'universe': 'TOP3000',
        'delay': 1,
        'decay': 5,
        'neutralization': 'INDUSTRY',
        'truncation': 0.05,
        'pasteurization': 'ON',
        'unitHandling': 'VERIFY',
        'nanHandling': 'OFF',
        'language': 'FASTEXPR',
        'visualization': False
    },
    'regular': '''
        
    '''
}